<a href="https://colab.research.google.com/github/julianVelandia/RedesNeuronalesConPyTorch/blob/master/redes-neuronales-recurrentes/5_2_Redes_neuronales_Recurrentes_con_PyTorch_(Colab)_FULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Redes Neuronales Recurrentes (RNNs) con PyTorch**

Las Redes Neuronales Recurrentes (RNN) son una arquitectura de red diseñada para procesar datos secuenciales, como texto o series temporales, donde el orden importa. A diferencia de las redes tradicionales, las RNN utilizan un estado oculto que se actualiza en cada paso, permitiendo que la red "recuerde" información previa en la secuencia.


Configuración del Entorno
Antes de empezar, asegúrate de tener instalado PyTorch, numpy y matplotlib. Estos son los paquetes principales que utilizaremos.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

## Creación de una secuencia de datos simple

Comenzamos importando las librerías que utilizaremos a lo largo del tutorial. `numpy` es útil para operaciones numéricas, `pandas` para el manejo de datos tabulares, y `torch` junto con sus submódulos nos permitirán construir y entrenar redes neuronales recurrentes (RNN) en PyTorch.

In [ ]:
data = [i for i in range(10)]
df = pd.DataFrame(data, columns=['value'])
print("Datos: ",df.head())

Datos:     value
0      0
1      1
2      2
3      3
4      4


Aquí generamos una secuencia de datos simple que va de 0 a 9. Luego, almacenamos estos valores en un DataFrame para visualizar los datos. Aunque estos datos son básicos, nos servirán para entender el funcionamiento de una RNN en un contexto sencillo.

In [ ]:
sequence = torch.tensor(data, dtype=torch.float32).view(-1, 1, 1)
print("Secuencia como Tensor: ", sequence)

Secuencia como Tensor:  tensor([[[0.]],

        [[1.]],

        [[2.]],

        [[3.]],

        [[4.]],

        [[5.]],

        [[6.]],

        [[7.]],

        [[8.]],

        [[9.]]])


## Red Neuronal Recurrente simple

Convertimos nuestra secuencia de datos en un tensor de PyTorch. Este tensor tendrá la forma adecuada para ser procesado por la RNN: `(longitud de la secuencia, tamaño del lote, número de características)`.

In [ ]:
class SimpleRNN(nn.Module):
    """Red Neuronal Recurrente simple."""

    def __init__(self, input_size, hidden_size, output_size):
        """Inicializa la red RNN."""
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        """Define el paso hacia adelante."""
        out, hidden = self.rnn(x, hidden)
        # primera fila: todos
        # Segunda fila: Último elemento (último paso temporal)
        # Tercera fila: todos
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        """Inicializa el estado oculto."""
        return torch.zeros(1, batch_size, self.hidden_size)

## Inicializando la red y los parámetros

Aquí definimos una clase `SimpleRNN` que representa una red neuronal recurrente simple. Esta clase incluye una capa RNN y una capa lineal que procesa la salida de la última celda RNN. Además, el método `init_hidden` inicializa el estado oculto a ceros.

In [ ]:

input_size = 1
hidden_size = 5
output_size = 1
rnn = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(rnn.parameters(), lr=0.01)

Inicializamos nuestra red neuronal con un tamaño de entrada de 1, un tamaño de capa oculta de 5, y un tamaño de salida de 1. Usamos el error cuadrático medio (`MSELoss`) como función de pérdida y el método de descenso de gradiente estocástico (`SGD`) como optimizador.

## Preparación de los datos para entrenamiento

Aquí preparamos los datos de entrenamiento. Usamos la misma secuencia como objetivo (`target`), lo que significa que la red intentará predecir cada valor de la secuencia dado el valor anterior.

In [ ]:
target = sequence.clone()
print("Objetivo: ", target)

Objetivo:  tensor([[[0.]],

        [[1.]],

        [[2.]],

        [[3.]],

        [[4.]],

        [[5.]],

        [[6.]],

        [[7.]],

        [[8.]],

        [[9.]]])


## Entrenamiento de la RNN con un bucle de entrenamiento básico

In [ ]:
sequence = sequence.view(10, 1, 1)

epochs = 10
for epoch in range(epochs):
    hidden = rnn.init_hidden(10)
    optimizer.zero_grad()
    output, hidden = rnn(sequence, hidden)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()


    print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 29.301565170288086
Epoch 1, Loss: 26.031265258789062
Epoch 2, Loss: 23.220966339111328
Epoch 3, Loss: 20.802444458007812
Epoch 4, Loss: 18.759593963623047
Epoch 5, Loss: 17.074010848999023
Epoch 6, Loss: 15.671239852905273
Epoch 7, Loss: 14.490802764892578
Epoch 8, Loss: 13.494032859802246
Epoch 9, Loss: 12.651274681091309


Entrenamos la red neuronal recurrente con un bucle básico. Durante cada época, inicializamos el estado oculto, calculamos la salida de la red, evaluamos la pérdida, retropropagamos los gradientes y actualizamos los parámetros de la red. Imprimimos la pérdida cada 10 épocas para monitorear el proceso de entrenamiento.

## Evaluación del modelo entrenado

In [ ]:
with torch.no_grad():
    hidden = rnn.init_hidden(10)
    predicted, hidden = rnn(sequence.view(10, 1, 1), hidden)
    print(predicted)
print("predicted: ", predicted)

tensor([[1.8678],
        [2.2586],
        [2.5121],
        [2.6620],
        [2.7442],
        [2.7849],
        [2.8011],
        [2.8037],
        [2.7993],
        [2.7918]])
predicted:  tensor([[1.8678],
        [2.2586],
        [2.5121],
        [2.6620],
        [2.7442],
        [2.7849],
        [2.8011],
        [2.8037],
        [2.7993],
        [2.7918]])


Después de entrenar la red, evaluamos su rendimiento utilizando los mismos datos de entrada. Esto nos permite observar qué tan bien la red ha aprendido a predecir la secuencia.